In [1]:
!pip install cvxopt
!pip install networkx matplotlib


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [2]:
!pip install pulp

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [3]:
import numpy as np
import pandas as pd
import cvxpy as cp
from IPython.display import display, Markdown
import logging

# ----------------- Initialization -----------------

def load_cost_matrix(url="https://raw.githubusercontent.com/Ajodo-Godson/TSP/main/backend/cost_matrix.csv"):
    try:
        C = pd.read_csv(url, header=None).values
        return C
    except Exception as e:
        logging.error(f"Failed to load cost matrix: {e}")
        raise

# ----------------- Data Definition -----------------

locations = [
    "Berlin Residence, Berlin, Germany",          # 0
    "Pier 39, San Francisco, CA",                 # 1
    "Salesforce Park, San Francisco, CA",         # 2
    "Bob's Doughnuts, San Francisco, CA",         # 3
    "851 California St, San Francisco, CA",       # 4
    "Union Square, San Francisco, CA",            # 5
    "San Francisco International Airport (SFO)",   # 6
    "Berlin TV Tower, Berlin, Germany",           # 7
    "Berlin Zoological Garden, Berlin, Germany",  # 8
    "Tempelhofer Feld, Berlin, Germany",          # 9
    "East Side Gallery, Berlin, Germany",         # 10
    "Berlin Brandenburg Airport (BER)"             # 11
]

coordinates = [
    (52.5200, 13.4050),  # Berlin Residence
    (37.8087, -122.4098), # Pier 39
    (37.7894, -122.3964), # Salesforce Park
    (37.7763, -122.4233), # Bob's Doughnuts
    (37.7924, -122.4070), # 851 California St
    (37.7880, -122.4075), # Union Square
    (37.6213, -122.3790), # SFO
    (52.5208, 13.4094),   # Berlin TV Tower
    (52.5086, 13.3372),   # Berlin Zoological Garden
    (52.4730, 13.4030),   # Tempelhofer Feld
    (52.5050, 13.4390),   # East Side Gallery
    (52.3667, 13.5033)    # BER
]

# Number of locations
N = len(locations)

# indices for specific locations
BER_residence = 0
SFO_airport = 6
BER_airport = 11

# ----------------- Cost Matrix Adjustment -----------------

def adjust_cost_matrix(C, penalty_edges=None, penalty_weight=1e4):
    """
    Adjust the cost matrix by adding penalties to specific edges.

    Args:
        C (np.ndarray): Original cost matrix.
        penalty_edges (list of tuples): List of edges to penalize.
        penalty_weight (float): The penalty weight to add.

    Returns:
        np.ndarray: Adjusted cost matrix.
    """
    if penalty_edges is None:
        penalty_edges = []
    C_adjusted = C.copy()
    for (i, j) in penalty_edges:
        C_adjusted[i, j] += penalty_weight
    return C_adjusted

# Penalty edges (SFO <-> BER)
penalty_edges = [
    (SFO_airport, BER_airport),
    (BER_airport, SFO_airport)
]

# Load and adjust the cost matrix
C = load_cost_matrix()
C = adjust_cost_matrix(C, penalty_edges=penalty_edges, penalty_weight=1e4)

# ----------------- Optimization Model -----------------

def define_tsp_model(C, N):
    # Define TSP Variables
    X = cp.Variable((N, N), boolean=True)
    U = cp.Variable(N, integer=True)

    # Objective: Minimize the total travel cost
    objective = cp.Minimize(cp.sum(cp.multiply(C, X)))

    return X, U, objective

def define_constraints(X, U, N):
    constraints = []

    # Each city must be entered and exited exactly once
    for i in range(N):
        constraints.append(cp.sum(X[:, i]) == 1)  # Enter each city once
        constraints.append(cp.sum(X[i, :]) == 1)  # Exit each city once

    # No self-loops for non-airport locations
    for i in range(N):
        if i not in [SFO_airport, BER_airport]:
            constraints.append(X[i, i] == 0)

    # Ensure the tour starts and ends at the Berlin Residence
    constraints.append(cp.sum(X[BER_residence, :]) == 1)
    constraints.append(cp.sum(X[:, BER_residence]) == 1)

    # Subtour elimination constraints (MTZ formulation)
    for i in range(1, N):
        constraints.append(U[i] >= 2)
        constraints.append(U[i] <= N)
        for j in range(1, N):
            if i != j:
                constraints.append(U[i] - U[j] + N * X[i, j] <= N - 1)

    return constraints

def solve_tsp_problem(objective, constraints):
    prob = cp.Problem(objective, constraints)
    prob.solve(solver=cp.GLPK_MI, verbose=True, max_iters=10000)
    return prob

def reconstruct_route(X_val, N):
    route = []
    current_node = BER_residence
    visited = set()

    while len(route) < N:
        route.append(current_node)
        visited.add(current_node)
        next_nodes = np.where(X_val[current_node] > 0.5)[0]
        if len(next_nodes) == 0:
            break
        next_node = next_nodes[0]
        if next_node in visited:
            break
        current_node = next_node

    # Ensure the route returns to the starting point
    if route[0] != route[-1]:
        route.append(route[0])

    return route

# ----------------- Solve TSP Function -----------------

def solve_tsp(C):
    X, U, objective = define_tsp_model(C, N)
    constraints = define_constraints(X, U, N)
    prob = solve_tsp_problem(objective, constraints)

    if prob.status == cp.OPTIMAL:
        X_val = X.value
        route_indices = reconstruct_route(X_val, N)
        route_names = [locations[i] for i in route_indices]
        route_coords = [coordinates[i] for i in route_indices]
        return prob.value, route_names, route_coords
    else:
        return None, [], []

# ----------------- Display Function -----------------

def display_route(optimal_value, optimal_route, optimal_coords):
    if optimal_value is not None:
        display(Markdown(f"## Optimal Value: {optimal_value:.2f}"))
        display(Markdown("### Optimal Route:"))
        
        # Create a DataFrame for better visualization
        route_data = {
            "Step": list(range(1, len(optimal_route) + 1)),
            "Location": optimal_route,
            "Coordinates": optimal_coords
        }
        df = pd.DataFrame(route_data)
        
        # Display as a styled table
        display(df.style.set_properties(**{
            'text-align': 'left',
            'padding': '10px'
        }).set_table_styles([
            {'selector': 'th', 'props': [('background-color', '#f2f2f2'), ('color', '#333'), ('padding', '12px 15px')]},
            {'selector': 'td', 'props': [('padding', '8px 15px')]}
        ]))
    else:
        display(Markdown("**No optimal solution found.**"))

# ----------------- Main Execution -----------------

if __name__ == "__main__":
    try:
        optimal_value, optimal_route, optimal_coords = solve_tsp(C)
        display_route(optimal_value, optimal_route, optimal_coords)
    except Exception as e:
        logging.error(f"Error during TSP solving: {e}")

                                     CVXPY                                     
                                     v1.6.0                                    
(CVXPY) Dec 13 12:47:22 PM: Your problem has 156 variables, 168 constraints, and 0 parameters.
(CVXPY) Dec 13 12:47:22 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Dec 13 12:47:22 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Dec 13 12:47:22 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Dec 13 12:47:22 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Dec 13 12:47:22 PM: Compiling problem (target solver=GLPK_MI)

## Optimal Value: 40087.17

### Optimal Route:

,Step,Location,Coordinates
0,1,"Berlin Residence, Berlin, Germany","(52.52, 13.405)"
1,2,"Berlin Zoological Garden, Berlin, Germany","(52.5086, 13.3372)"
2,3,"Berlin TV Tower, Berlin, Germany","(52.5208, 13.4094)"
3,4,"East Side Gallery, Berlin, Germany","(52.505, 13.439)"
4,5,"Tempelhofer Feld, Berlin, Germany","(52.473, 13.403)"
5,6,Berlin Brandenburg Airport (BER),"(52.3667, 13.5033)"
6,7,"Pier 39, San Francisco, CA","(37.8087, -122.4098)"
7,8,"Bob's Doughnuts, San Francisco, CA","(37.7763, -122.4233)"
8,9,"851 California St, San Francisco, CA","(37.7924, -122.407)"
9,10,"Union Square, San Francisco, CA","(37.788, -122.4075)"
